In [ ]:
### Import Libraries.

library("MiloR")
library("Seurat")
library("SingleCellExperiment")
library("scater")
library("scran")
library("dplyr")
library("ggplot2")
library("patchwork")
library("ggbeeswarm")
library("qs")

In [ ]:
### Load Data.

setwd("/folder/")
Seurat <- qread("Seurat.qs")

In [ ]:
### Subset Diagnostic Groups. 

Seurat_subset <- subset(Seurat, subset = Status %in% c("Control", "sALS")) ### Or C9ALS

In [ ]:
### Convert to SingleCellExperiment. 

sce <- as.SingleCellExperiment(Seurat_subset, assay = "RNA")
sce

In [ ]:
### Create Milo Object.

milo <- Milo(sce)

In [ ]:
### Graph Construction.

milo <- buildGraph(milo, k = 30, d = 40, reduced.dim = "PCA", transposed = TRUE)

In [ ]:
### Neighborhood Definition.

milo <- makeNhoods(milo, prop = 0.05, k = 30, d = 40, refined = TRUE, reduced_dims = "PCA")

In [ ]:
### Inspect Neighborhood Size.

plotNhoodSizeHist(milo) + scale_x_continuous(breaks = seq(0, 200, 50))

In [ ]:
### Counting and Metadata.

milo <- countCells(milo, meta.data = data.frame(colData(milo)), sample = "Sample_ID")

In [ ]:
### Design Matrix.

design_df <- colData(milo) %>%
  as.data.frame() %>%
  dplyr::select(Sample_ID, Status, Dataset) %>%
  distinct()
rownames(design_df) <- design_df$Sample_ID
design_df <- design_df[colnames(nhoodCounts(milo)), , drop = FALSE]

In [ ]:
### DA Testing.

milo <- calcNhoodDistance(milo, d = 40)
da_results <- testNhoods(milo, design = ~ Status, design.df = design_df)

In [ ]:
### Filter by Significance.

da_results_filtered <- subset(da_results, SpatialFDR < 0.05)

In [ ]:
### Nhood Plot.

milo <- buildNhoodGraph(milo, overlap = 2)

p_umap <- plotUMAP(milo, colour_by = "Superclusters") +
  plotNhoodGraphDA(milo, da_results, alpha = 0.05) +
  plot_layout(guides = "collect")

In [ ]:
### Beeswarm Plot by Cluster Group.

da_results <- annotateNhoods(milo, da_results, coldata_col = "Cluster_Column")
plotDAbeeswarm(da_results, group.by = "Cluster_Column", alpha = 0.05)

In [ ]:
### Beeswarm Plot Paper.

da_results_filtered$Status <- factor(
  da_results_filtered$Status,
  levels = c("Control", "sALS", "Mixed") ### C9ALS.
)

da_results_filtered$sig_group <- ifelse(
  da_results_filtered$FDR < 0.05,
  as.character(da_results_filtered$Status),
  "nonsig"
)

da_results_filtered$sig_group <- factor(
  da_results_filtered$sig_group,
  levels = c("Control", "sALS", "Mixed", "nonsig")
)

plot <- ggplot(
  da_results_filtered,
  aes(x = Superclusters, y = logFC, color = sig_group)
) +
  geom_quasirandom(alpha = 0.8, size = 2) +
  scale_color_manual(
    values = c(
      "Control" = "#ADB17DFF",
      "sALS"    = "#5B8FA8FF",
      "Mixed"   = "grey60",
      "nonsig"  = "grey85"
    )
  ) +
  coord_flip() +
  theme_minimal() +
  labs(
    title = "Differential Abundance Beeswarm Plot",
    y = "log2 Fold Change",
    x = "Superclusters",
    color = "Group"
  ) +
  theme(
    axis.text.y = element_text(hjust = 1, size = 10),
    axis.text.x = element_text(size = 10),
    plot.title = element_text(size = 14, face = "bold"),
    legend.position = "right"
  )

plot

In [ ]:
### Markers of DA Neighborhoods.

milo <- logNormCounts(milo)
da_results$NhoodGroup <- as.numeric(da_results$SpatialFDR < 0.05 & da_results$logFC > 0)
nhood_markers <- findNhoodGroupMarkers(milo, da_results)
head(nhood_markers[order(-nhood_markers$logFC_0), ], 10)

In [ ]:
### Group DA Neighborhoods for Summary.

da_results <- groupNhoods(milo, da_results, da.fdr = 0.05, overlap = 1)